# Imports

In [1]:
import os
import sys
import pathlib

import cv2
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

import torchvision
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt 
import albumentations as albu

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

# Fix to be able to import python modules inside a notebook
os.chdir('..')

# Useful extensions
# %load_ext watermark
# %watermark -v -n -m -p numpy,torch,albumentations,photosynthesis_metrics

%load_ext autoreload
%autoreload 2

# Nice plot formating
%matplotlib inline

# Code

# DBA

In [4]:
from loguru import logger
from src.callbacks import cmc_score_count, rank_map_score

In [5]:
CONFIG_PATH = pathlib.Path("logs/resnet101_384_arcface_fp16_gem_light/")
# Val: Acc@1 0.81258, CMC@10 0.92955, mAP@10 0.80109, target 0.80683. In reality it's 0.56
CONFIG_PATH = pathlib.Path("logs/genet_normal_384_arcface_fp16_gem_light/") 
df = pd.read_csv(CONFIG_PATH / "train_val.csv")

In [9]:
val_embeddings = torch.tensor(list(map(eval, df["embeddings"].values)))

In [89]:
top_k = 10
topk_vals, topk_ind = distances.neg().topk(top_k, dim=1)
cosine_dist = (2 - topk_vals.neg()) * 0.5 # cos = ((2 - l2_distances) / 2)
cosine_weight = (cosine_dist ** alpha)[..., None] # N x TOPK -> N x TOPK x 1
# new_embedding = val_embeddings[topk_ind].mean(dim=1)
(val_embeddings[topk_ind] * cosine_weight).mean(dim=1)

In [8]:
def data_based_augmentation(embeddings, top_k=10, alpha=2, include_self=False):
    """
    Args:
        embeddings (Tensor): original embeddings
        top_k (int): How many neighbours to use
        alpha (int): Power for 
    """
    # Matrix of pairwise cosin distances
    distances = torch.cdist(val_embeddings, val_embeddings)
    # Nearest neighbours
    topk_vals, topk_ind = distances.neg().topk(top_k, dim=1)
    # Get weight
    cosine_dist = (2 - topk_vals.neg()) * 0.5 # cos = ((2 - l2_distances) / 2)
    cosine_weight = (cosine_dist ** alpha)[..., None] # N x TOPK -> N x TOPK x 1
    new_embedding = val_embeddings[topk_ind] * cosine_weight # N x TOPK x EMBED_SIZE * N x TOPK x 1
    if include_self:
        new_embedding = new_embedding.mean(dim=1)
    else:
        new_embedding = new_embedding[:, 1:].mean(dim=1)
    return new_embedding